# **What are we going to do:**

1. Craw data poemtry from internet and saved
2. Preprocessing data, and write **DataLoader** ( return tokenizer of poemtry)
3. Initialize Trainer with TrainingArguments and GPT-2 model
4. Train and save the model
5. test the model

 *CHECK CUDA, INSTALL LIBRARY NECCESSARY*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/BERT'
os.chdir(path)
!ls

68cleanregex.txt
68cleanvl_test.txt
cached_lm_PhobertTokenizer_254_68cleanregex.txt
cached_lm_PhobertTokenizer_254_68cleanregex.txt.lock
cached_lm_PhobertTokenizer_254_68cleanvl_test.txt
cached_lm_PhobertTokenizer_254_68cleanvl_test.txt.lock
gpt2-poem
sample.txt
save_modelGPT2
Untitled0.ipynb


In [ ]:
import torch
torch.__version__
torch.cuda.is_available()

False

In [ ]:
!pip install transformers
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp

     |████████████████████████████████| 1.4MB 6.0MB/s 
     |████████████████████████████████| 2.9MB 31.6MB/s 
     |████████████████████████████████| 890kB 28.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=449d5aed50317afb94536a58cfcf6541820b2939143657ce70537ea6f89eb379
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.7MB 5.8MB/s 
     |████████████████████████████████| 122kB 35.6MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 112kB 24.0MB/s 
     |████████████████████████████████| 276kB 41.9MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141231 sha256=ece4e6914d9bacc41931e4520c919086466cb3ba9fea34816f57d40bf21e0529
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf39

# **Step1 : Craw data in internet and save 2 file train, test**

In [ ]:
train_path = '/content/drive/MyDrive/BERT/68cleanregex.txt'
test_path = '/content/drive/MyDrive/BERT/68cleanvl_test.txt'

# 2 direct load Pretrain Model PhoBERT :
( load pretrain model PhoBERT to preprocessing data => write dataLoader)

1.   Use fairseq
2.   Use transformers



# Case 1 : Load pretrainModel PhoBERT use fairseq

In [ ]:
# #--------------Load the model in fairseq-----------------#
# from fairseq.models.roberta import RobertaModel
# phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')

# #-------------------BPE encoding -----------------------#
# from fairseq.data.encoders.fastbpe import fastBPE
# # Khởi tạo Byte Pair Encoding cho PhoBERT
# class BPE():
#   bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'
# args = BPE()
# #----------Incorporate the BPE encoder into PhoBERT-----------#
# phoBERT.bpe = fastBPE(args) 
# #-------------encode --------------#
# tokens = phoBERT.encode('Debug la vấn đề thường ngày của 1 coder!!')
# print('tokens list : ', tokens)
# #-----------Decode ngược lại thành câu từ chuỗi index token---------------#
# phoBERT.decode(tokens)  # 'Hello world!'


# Case 2 : Load pretrainModel PhoBERT use transformer

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phoBERT = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
# ------add token ('\n') to enter lines --------#
custokenizer.add_tokens('\n')
# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Debug là việc thường xuyên của delevoper."

input_ids = torch.tensor([custokenizer.encode(line)])
with torch.no_grad():
    features = phoBERT(input_ids)  # Models outputs are now tuples


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


## **Step2 . Code DATALOADER**


*   Create dataset read file train, test :return tokenizers ( len =256)
*   Dataloader



In [ ]:
#--------------Create Dataset----------------#
import os
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from filelock import FileLock
from transformers.utils import logging
from typing import Dict, List, Optional
import pickle
import random
import time
logger = logging.get_logger(__name__)

class PoemDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    Parameters:
    ----------
    tokenizers : is pretrain tokenizer of PhoBERT
    file_path  : path to file train, test
    block_size : size of 1 block , optinal
    cache_dir  : just load 1 once and saved

    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"
        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # -----------Make sure only the first process in distributed training processes the dataset,----------------#
        # ---------------------------------------and the others will use the cache------------------------#
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    text = f.read()
                #-----convert text to tokenizers----------------------------#
                '''
                1. Convert word -> subword (tokenizer.tokenize(text))
                2. COnvert subword -> number (tokenizer.convert_tokens_to_ids)
                '''
                tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

                # ------------- Truncate in block of block_size-----------------#
                #-----------Beacuse add_token('\n') -> inds = 64001------------#
                #--------If len(block_size)>256 so cut and add_special_tokens (<s>, </s>)---------------#
                i = 0
                while i < len(tokenized_text) - block_size + 1:
                    inds = tokenized_text[i : i + block_size]
                    for j in range(0, len(inds)):
                        if inds[j]==64001:
                            inds = inds[j+1:] #remove the first \n
                            break
                    for j in range(len(inds)-1, 0, -1):
                        if inds[j]==64001:
                            inds = inds[:j-1] #remove \n
                            break
                    i += len(inds)
                    self.examples.append(
                        tokenizer.build_inputs_with_special_tokens(inds)
                    )
                    
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should loook for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [ ]:
#-----------Load dataset-----------------------#
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling, LineByLineWithSOPTextDataset

def load_dataset(train_path, test_path, custokenizer):
    train_dataset = PoemDataset(
          tokenizer=custokenizer,
          file_path=train_path,
          block_size= 256)
     
    test_dataset = PoemDataset(
          tokenizer=custokenizer,
          file_path=test_path,
          block_size=256)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=custokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,custokenizer)

In [ ]:
#-----------Test dataloader----------------#
print(len(test_dataset))
print(len(train_dataset))
#-------------Test decode to sentence ---------------#
print(custokenizer.decode(test_dataset[1]))

52
1746
<s> trẻ trung trời đất cho ta 
 thời gian đòi lại thế là ước mong 
 mùa xuân như gái không chồng 
 có bao nhiêu vẫn là không có gì 
 người ơi hoa bẻ mang đi 
 để mình ta với xanh rì trời xanh 
 
 ở đâu chỉ những mộng êm 
 ở đâu chắc chắn xây lên mặn mà 
 đó là bến đậu của ta 
 đó là tiếng hát đó là tứ thơ 
 đó là chỗ hết vẩn vơ 
 để ta mãi mãi đáng chờ đợi nhau 
 
 nơi đây bao chàng ra đi 
 để cho ai đặt bến ni không chồng 
 cuộc chiến tranh chẳng chờ mong 
 biết bao trai tráng xung phong lên đường 
 hy sinh tại chốn chiến trường 
 để người con gái nhớ thương trọn đời 
 bao ngày nước mắt tuôn rơi 
 hóa thành bia đá giữa trời mùa thu 
 trở thành câu hát lời </s>


# **Step 3. Initialize Trainer with TrainingArguments and GPT-2 model**

In [ ]:
from transformers import Trainer, TrainingArguments, GPT2Config, GPT2LMHeadModel

In [ ]:

#--------------------------Load  pretrain model GPT-2--------------------#
model_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
# vntq.save_pretrained('/content/drive/MyDrive/BERT/save_modelGPT2/')


In [ ]:
model_gpt2.config

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "use_cache": true,
  "vocab_size": 50257
}

In [ ]:
model_gpt2

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
# Check linear layer final of gpt2
model_gpt2.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [ ]:
# Check weights of linear layer final
model_gpt2.lm_head.weight

Parameter containing:
tensor([[-0.1101, -0.0393,  0.0331,  ..., -0.1364,  0.0151,  0.0453],
        [ 0.0403, -0.0486,  0.0462,  ...,  0.0861,  0.0025,  0.0432],
        [-0.1275,  0.0479,  0.1841,  ...,  0.0899, -0.1297, -0.0879],
        ...,
        [-0.0445, -0.0548,  0.0123,  ...,  0.1044,  0.0978, -0.0695],
        [ 0.1860,  0.0167,  0.0461,  ..., -0.0963,  0.0785, -0.0225],
        [ 0.0514, -0.0277,  0.0499,  ...,  0.0070,  0.1552,  0.1207]],
       requires_grad=True)

In [ ]:
# Random weights => fine-turning model
rand_weight = torch.rand(model_gpt2.lm_head.weight.shape)
print(rand_weight)

tensor([[0.5111, 0.7020, 0.7175,  ..., 0.7280, 0.4061, 0.9498],
        [0.0895, 0.8086, 0.6039,  ..., 0.4003, 0.9907, 0.0363],
        [0.3005, 0.7193, 0.2726,  ..., 0.4621, 0.2552, 0.2104],
        ...,
        [0.4353, 0.6130, 0.0160,  ..., 0.6971, 0.4949, 0.5223],
        [0.2368, 0.6301, 0.1006,  ..., 0.7782, 0.7717, 0.5698],
        [0.1846, 0.6800, 0.6224,  ..., 0.4731, 0.2500, 0.9200]])


In [ ]:
model_gpt2.lm_head.weight = torch.nn.parameter.Parameter(rand_weight)

In [ ]:
model_gpt2.lm_head.weight

Parameter containing:
tensor([[0.5111, 0.7020, 0.7175,  ..., 0.7280, 0.4061, 0.9498],
        [0.0895, 0.8086, 0.6039,  ..., 0.4003, 0.9907, 0.0363],
        [0.3005, 0.7193, 0.2726,  ..., 0.4621, 0.2552, 0.2104],
        ...,
        [0.4353, 0.6130, 0.0160,  ..., 0.6971, 0.4949, 0.5223],
        [0.2368, 0.6301, 0.1006,  ..., 0.7782, 0.7717, 0.5698],
        [0.1846, 0.6800, 0.6224,  ..., 0.4731, 0.2500, 0.9200]],
       requires_grad=True)

In [ ]:
'''
Because GPT2 has vocabulary_size 50257 and (wte): Embedding(50257, 768)
So  convert vocabulary_size= 64002, Embedding(64002, 768)
'''
task_gpt2 = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size
config_gpt2 = configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
                           task_specific_params=task_gpt2,
                           eos_token_id = 2,
                           bos_token_id = 0,
                           pad_token_id = 1,
                           sep_token_id = 2,
                          #  eos_token_id=custokenizer.eos_token_id,
                          #  bos_token_id=custokenizer.bos_token_id, 
                          #  pad_token_id=custokenizer.pad_token_id,
                          #  sep_token_id=custokenizer.sep_token_id
                           )
model_gpt2 = GPT2LMHeadModel(config_gpt2)
model_gpt2


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64002, 768)
    (wpe): Embedding(258, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Layer

In [ ]:
# Check model gp2 custormer
model_gpt2.config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 258,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 258,
  "pad_token_id": 1,
  "resid_pdrop": 0.1,
  "sep_token_id": 2,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 256
    }
  },
  "use_cache": true,
  "vocab_size": 64002
}

In [ ]:
#save model_gpt2 (vocabulary_size =64002)
model_gpt2.save_pretrained('/content/drive/MyDrive/BERT/save_modelGPT2/')

## NEW **MODEL**

In [ ]:
task = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size

In [ ]:
configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
                           task_specific_params=task,
                           eos_token_id = 2,
                           bos_token_id = 0,
                           pad_token_id = 1,
                           sep_token_id = 2,
                          #  eos_token_id=custokenizer.eos_token_id,
                          #  bos_token_id=custokenizer.bos_token_id, 
                          #  pad_token_id=custokenizer.pad_token_id,
                          #  sep_token_id=custokenizer.sep_token_id
                           )
poem = GPT2LMHeadModel(configuration)


In [ ]:
poem

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64002, 768)
    (wpe): Embedding(258, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Layer

In [ ]:
# Load weights of model_gpt2 ( random weights)
load_model_gpt2 = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/BERT/save_modelGPT2/')
poem.load_state_dict(load_model_gpt2.state_dict())

<All keys matched successfully>

In [ ]:
#-----------Print process training ------------#
from transformers.trainer_callback import TrainerCallback
from transformers import pipeline
class PrinterCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        if int(state.epoch)%10==0:
            pipe = pipeline('text-generation', model=model, tokenizer=custokenizer, device=0)
            with open("/content/drive/MyDrive/BERT/sample.txt", "a") as f:
                f.write(pipe('<s> tìm về một thuở hạ xưa')[0]['generated_text'])
                f.write("\n===========================================\n")
                f.close()

In [ ]:

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/BERT/gpt2-poem", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=8, # batch size for training  
    per_device_eval_batch_size=16,  # batch size for evaluation
    save_steps=5000, # after # steps model is saved 
    save_total_limit = 2, # delete other checkpoints
    warmup_steps=500,    # number of warmup steps for learning rate scheduler
    logging_dir='/content/drive/MyDrive/BERT/gpt2-poem/logs', # directory for storing logs
    logging_steps=500,
    )

## **Step 4. TRAIN MODEL**

In [ ]:
device = torch.device('cuda')
trainer = Trainer(
    model=poem, # GPT2
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks = [PrinterCallback],

)

In [ ]:
# -------Train and save model-----------#
trainer.train()
trainer.save_model()

Step,Training Loss


## **Step 4. TEST MODEL**

In [ ]:
#-------Load model saved-----------------#
from transformers import pipeline
poem = pipeline('text-generation', model="/content/drive/MyDrive/BERT/gpt2-poem", tokenizer=custokenizer, config={'max_length':1024})

In [ ]:
#Test
a = poem('chiều')


In [ ]:
x = a[0]['generated_text']
print(x)
# print(a[0]['generated_text'])

chiều thu em về phía mái đình 
 dáng thon giữa dưới long lanh tháng giêng 
 trao nhau quyện nhĩ lạnh áo hồng 
 tóc dài tóc trắng phiêu diêu bông 
 nhịp buồn sao cứ nhẹ nhàng 
 chung tay anh vội vã à ơi à 
 âm khờ một thủa nương nuôi 
 tu nào cũng chẳng quản chi nào ngờ 
 
 mấy mươi năm ấy bon chen 
 em là anh đến bím tóc mưa nguồn 
 nắng mưa gom gió nhăn gà 
 trong nhà bên đập chùm đồng cân mần 
 anh đi tìm lấy chồng gà 
 làm cho nhẹ hắt buông mành thời gian 
 
 chị bồng con nít nhà nghèo 
 cái chua rách nát nhàu bến bờ ao 
 còn e ấp ủ cho trầu 
 cứ lời ru lại cơn mưa cuối trời 
 sao em run mục đồng xanh 
 chiếc ba lô rách với vài lá trầu 
 không mưa gió giật mình vừa 
 cho tuổi thơ tóc thả dần vơi dày 
 
 dòng sông hóa bóng mây che 
 đợi chờ đợi gió níu mây thưa 
 gịp rượi rụng gió giông tàn 
 tháng tư nhịp sóng giữa dòng mương cuối làng 
 rộng nước chảy chưa kịp nương 
 tìm đâu thấy có vui này dệt duyên 
 thanh cao gặp lúc lấy chồng 
 gieo thêm mỗi đứa đi tìm
